In [ ]:
import glob
import lzma
import os
import pickle
import sys

import pandas
from pandas import DataFrame, concat
from s2clientprotocol import sc2api_pb2 as sc_pb
from sc2.main import _play_replay, _setup_replay, get_replay_version
from sc2.sc2process import SC2Process
from tqdm import tqdm

sys.path.append("../ares-sc2")  # required to import sc2_helper
sys.path.append("../ares-sc2/src")  # required to import ares
sys.path.append("phantom")

from ares import ALL_STRUCTURES, WORKER_TYPES

from phantom.common.constants import UNIT_COUNTER_DICT
from phantom.replay import Replay, ReplayBot, ReplayStep, Report

In [ ]:
sc_pb.ObserverAction

In [ ]:
sc_pb.__dict__

In [ ]:
pandas.set_option("plotting.backend", "plotly")

In [ ]:
PLAYERS = [0, 1, 2]

UNIT_TYPES = WORKER_TYPES | ALL_STRUCTURES | UNIT_COUNTER_DICT.keys()


def state_to_df(game_loop: int, state: ReplayStep) -> DataFrame:
    compositions = state.player_compositions()
    state.player_upgrades()
    records = []
    for player, composition in compositions.items():
        record = {"player": player, "game_loop": game_loop}
        record.update({u.name: composition.get(u.name, 0) for u in UNIT_TYPES})
        records.append(record)
    return DataFrame(records)

In [ ]:
observe_id = 2
realtime = False
for replay_file in glob.glob("../resources/replays/aiarena/*.SC2Replay"):
    bot = ReplayBot()
    replay = Replay.from_file(replay_file)
    base_build, data_version = get_replay_version(replay_file)
    async with SC2Process(fullscreen=True, base_build=base_build, data_hash=data_version) as server:
        client = await _setup_replay(server, os.path.abspath(replay_file), realtime, observe_id)
        result = await _play_replay(client, bot, realtime, observe_id)

In [ ]:
bot.replay.steps[4000].player_compositions()

In [ ]:
REPLAY_FILES = glob.glob("../resources/replays/local/*.pkl.xz")

In [ ]:
dfs = list[DataFrame]()

for replay_file in REPLAY_FILES:
    with lzma.open(replay_file, "rb") as f:
        report: Report = pickle.load(f)

    state_bot = report.replay_bot.steps[0]
    game_loops = sorted(set(report.replay_bot.steps.keys() | report.replay_observer.steps.keys()))
    for game_loop in tqdm(game_loops, desc=replay_file):
        if state_bot_update := (report.replay_bot.steps.get(game_loop)):
            state_bot = state_bot_update
        if state_observer_update := report.replay_observer.steps.get(game_loop):
            state_observer = state_observer_update
        df_bot = state_to_df(game_loop, state_bot)
        df_bot["seen"] = True
        dfs.append(df_bot)
        df_observer = state_to_df(game_loop, state_observer)
        df_observer["seen"] = False
        dfs.append(df_observer)

df_replay = concat(dfs).set_index(["game_loop", "player", "seen"])

In [ ]:
seen = df_replay.loc[(slice(None), 1, True), :].droplevel(["player", "seen"])
unseen = df_replay.loc[(slice(None), 1, False), :].droplevel(["player", "seen"])
(seen - unseen).plot()

In [ ]:
df_replay.loc[(slice(None), 2, False), "DRONE"].droplevel(["player", "seen"]).plot()

In [ ]:
df_replay.loc[(slice(None), 2, True), "DRONE"].droplevel(["player", "seen"]).plot()